In [2]:
import akshare as ak
import pandas as pd

In [9]:
code = '600746'
stock = ak.stock_zh_a_hist(symbol=code, 
                        period="daily",
                        start_date="20231001",
                        end_date='20250325', 
                        adjust="")

In [10]:
stock['MA10'] = stock['收盘'].rolling(window=10).mean()
stock['MA25'] = stock['收盘'].rolling(window=25).mean()
stock['V5'] = stock['成交量'].rolling(window=5).mean()
stock['V60'] = stock['成交量'].rolling(window=60).mean()
stock = stock[['日期','股票代码','开盘','收盘','最高','最低','成交量','振幅','涨跌幅','涨跌额','MA10','MA25','V5','V60']]
stock.tail(3)

,日期,股票代码,开盘,收盘,最高,最低,成交量,振幅,涨跌幅,涨跌额,MA10,MA25,V5,V60
352,2025-03-21,600746,7.36,7.35,7.46,7.33,37383,1.76,-0.54,-0.04,7.438,7.3212,36567.4,44010.633333
353,2025-03-24,600746,7.31,7.39,7.42,7.28,46524,1.90,0.54,0.04,7.436,7.3276,37260.2,44084.516667
354,2025-03-25,600746,7.36,7.69,7.77,7.36,169363,5.55,4.06,0.30,7.457,7.3504,63918.6,46305.750000


In [11]:
df = stock.copy()

In [12]:
# 入场买点
def buy(df):
    buy = []
    for i in range(65,len(df)-1):
        if all ([df['开盘'][i] < df['MA25'][i],\
                df['收盘'][i] > df['MA25'][i],\
                df['MA25'][i] > df['MA25'][i-5],\
                df['V5'][i]> df['V60'][i]]):
            buy.append(i)
        else:
            continue
    return buy

In [13]:
def sale(b):
    b_high = df['收盘'][b]*1.1 #盈利线
    b_low = df['最低'][b]*0.99 #止损线
    for w in range(b+1,len(df)):
        # 收盘价达到盈利线
        if df['最高'][w]>b_high:
            s = w
            b_high = df['最高'][w]
            b_low = (df['最高'][w]-df['收盘'][b])*0.7+df['收盘'][b]
            continue
        # 跌破止损价，卖出
        elif all([df['收盘'][w]<b_low,\
                 df['涨跌幅'][w]<-5,]):
            s = w
            break
        elif all([df['收盘'][w]<b_low,\
                 df['涨跌幅'][w]<-0.5,\
                 df['收盘'][w]<df['MA10'][w]]):
            s = w
            break
        else:
            s = w
            continue
    return s  

In [14]:
# 交易回测
money = 10000
b_dt = buy(df)
print(b_dt)
if not b_dt:
    print('无符合条件指标，请更换股票')
else:
    s_date = df['日期'][1]
    for k in range(len(b_dt)):
        b = b_dt[k]
        if df['日期'][b]>s_date:
            #print(str(df['日期'][b]) + ' | ' + str(df['CCI'][b]))
            b_sym = float(((money/df['收盘'][b])//100)*100)
            money = money-(b_sym*df['收盘'][b])
            print('买入： '+ str(df['日期'][b]) + ' | ' + str(df['收盘'][b]) + ' | ' + str(b_sym) + ' | ' + str(money))
            s = sale(b)
            money = money+(b_sym*df['收盘'][s])
            b_sym = 0
            s_date = df['日期'][s]
            print('买出： '+ str(df['日期'][s]) + ' | ' + str(df['收盘'][s]) + ' | '+ str(b_sym) + '     | ' + str(money))
        else:
            continue

[128, 129, 134, 136, 223, 224, 275]
买入： 2024-04-17 | 6.49 | 1500.0 | 265.0
买出： 2024-05-16 | 7.14 | 0     | 10975.0
买入： 2024-09-03 | 6.19 | 1700.0 | 452.0
买出： 2024-10-09 | 6.9 | 0     | 12182.0
买入： 2024-11-25 | 7.29 | 1600.0 | 518.0
买出： 2024-12-31 | 7.01 | 0     | 11734.0


##### 